**Задание 1.**
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [101]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
KEYWORDS = ['python', 'парсинг']

In [104]:

req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')


In [106]:
URLlist=[]
for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
            if ([hub_lower in desired for desired in KEYWORDS]):
                title_element = post.find('a', class_='post__title_link').attrs.get('href')
                URLlist.append(title_element)
                break

In [111]:
def get_habr_posts(links):
    habr_posts = pd.DataFrame()
    for link in links:
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        date=soup.find('span',class_='post__time')    
        title = soup.find('span', class_='post__title-text').text   
        if soup.find('div', class_='post__text post__text_v2'): 
            text = soup.find('div', class_='post__text post__text_v2').text
        elif soup.find('div', class_='post__text post__text-html post__text_v1') :
            text = soup.find('div', class_='post__text post__text-html post__text_v1').text   
        row = {'date': date, 'title': title,'URL':link, 'text': text}
        habr_posts = pd.concat([habr_posts, pd.DataFrame([row])])  
    return habr_posts

habr_posts = get_habr_posts(URLlist)
habr_posts

,date,title,URL,text
0,[сегодня в 21:50],[Личный опыт] Фронтенд-инженер из лондонского ...,https://habr.com/ru/company/gms/blog/526366/,"Как готовиться к собеседованиям, чтобы устроит..."
0,[сегодня в 21:43],Цена естественности или как обогнать QuickSort,https://habr.com/ru/post/526346/,Сортировка — такая же «вечная» тема для алгори...
0,[сегодня в 21:11],Flutter для React/React Native разработчика,https://habr.com/ru/post/526362/,"Статья просвещена тем, кто пишет на стеке Reac..."
0,[сегодня в 20:42],Как автоматизировать аудит всех разговоров ком...,https://habr.com/ru/post/526360/,\nРазговоры с клиентами записывают практически...
0,[сегодня в 20:30],Мультимедиа прошлого: как слушали музыку в MS-DOS,https://habr.com/ru/company/selectel/blog/526328/,\r\nВсего каких-то 26 лет назад прослушивать м...
0,[сегодня в 20:10],Истоки виртуализации,https://habr.com/ru/company/it-grad/blog/526326/,"Внимание, вопрос: кто стоял у истоков виртуали..."
0,[сегодня в 20:01],Большой тест батареек Крона,https://habr.com/ru/company/lamptest/blog/526356/,Весной я протестировал 9 батареек типа «Крона»...
0,[сегодня в 19:41],ModBus Slave RTU/ASCII без смс и регистрации,https://habr.com/ru/post/526350/,\n\r\nВ открытом доступе присутствует множеств...
0,[сегодня в 19:22],"10 высокотехнологичных продуктов, которые мы б...",https://habr.com/ru/post/526344/,"Ученые и футуристы настаивают на том, что из-з..."
0,[сегодня в 18:57],«Как жить без стрелок» или не так страшен сплит…,https://habr.com/ru/post/526338/,"""Как жить без стрелок"" или не так страшен спли..."
